In [1]:
import numpy as np
from scipy.spatial.distance import euclidean
import scipy
import torch

import dtw
import dtwalign
from fastdtw import fastdtw

from sklearn.manifold import TSNE
import pandas as pd

from src.speech_classification.audio_processing import AudioPreprocessorFbank, SpeechCommandsDataCollector
from src.siamese_net_sound_similarity.slstm_train import SiameseSpeechCommandsDataCollector, StochasticSiameseLSTMNet
from src.siamese_net_sound_similarity.train import SiameseLSTMNet


import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set()

%matplotlib inline

In [2]:
DISTANCE='canberra'

## Load data

In [3]:
wanted_words = ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left',
                'marvin',
                'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two',
                'up', 'wow', 'yes', 'zero']

wanted_words_combined = wanted_words

model_settings = {
    'dct_coefficient_count': 26,
    'label_count': len(wanted_words_combined) + 2,
    'hidden_reccurent_cells_count': 128,
    'winlen': 0.04,
    'winstep': 0.02
}

In [4]:
preproc = AudioPreprocessorFbank(nfilt=model_settings['dct_coefficient_count'], winlen=model_settings['winlen'],
                                     winstep=model_settings['winstep'])

data_iter = SiameseSpeechCommandsDataCollector(preproc,
                                        data_dir=r'C:\Study\Speech_command_classification\speech_dataset',
                                        wanted_words=wanted_words_combined,
                                        testing_percentage=10,
                                        validation_percentage=10
                                        )

index_to_word = {v:k for k,v in data_iter.word_to_index.items()}

In [5]:
n_mini_batch_size = 1
data = data_iter.get_data(n_mini_batch_size, 0, 'training')
labels = data['y']

duplicates = data_iter.get_duplicates(labels, 0, 'training')
assert np.any(labels == duplicates['y'])

non_duplicates = data_iter.get_nonduplicates(labels, 0, 'training')
assert np.any(labels != non_duplicates['y'])

In [ ]:

i = np.random.randint(data['y'].shape[0])
sample_idx = i

fig, axes = plt.subplots(2, 3, figsize=(15,10))

# plt.figure()
im = axes[0][0].imshow(data['x'][i].T)
# fig.colorbar(im)
label = index_to_word[data['y'][0]]
axes[0][0].title.set_text(f'{label}')

axes[0][1].imshow(duplicates['x'][i].T)
# plt.colorbar()
label = index_to_word[data['y'][0]]
axes[0][1].title.set_text(f'{label}')


axes[0][2].imshow(non_duplicates['x'][i].T)
# plt.colorbar()
label = index_to_word[non_duplicates['y'][0]]
axes[0][2].title.set_text(f'{label}')

######################################## DTW

x = data['x'][i]

# self dtw (extpected to be 0)

y = data['x'][i]
res = dtwalign.dtw(x, y, dist=DISTANCE, open_end=True)
res.plot_path(ax=axes[1][0])
axes[1][0].set_title(f"anchor DTW = {res.normalized_distance:.4f}")


# positive dtw

y = duplicates['x'][sample_idx]
res = dtwalign.dtw(x, y, dist=DISTANCE, open_end=True)
res.plot_path(ax=axes[1][1])
axes[1][1].set_title(f"positive DTW = {res.normalized_distance:.4f}")


# negative dtw

y = non_duplicates['x'][sample_idx]
res = dtwalign.dtw(x, y, dist=DISTANCE, open_end=True)
res.plot_path(ax=axes[1][2])
axes[1][2].set_title(f"negative DTW = {res.normalized_distance:.4f}")

# Gather Data for analysis

In [ ]:
n_mini_batch_size = 3000

# gather data


data = data_iter.get_data(n_mini_batch_size, 0, 'training')
labels = data['y']

duplicates = data_iter.get_duplicates(labels, 0, 'training')
assert np.any(labels == duplicates['y'])

non_duplicates = data_iter.get_nonduplicates(labels, 0, 'training')
assert np.any(labels != non_duplicates['y'])


## DTW distribution on raw data

In [ ]:

# initialize dist_lists

duplicates_dtw = []
non_duplicates_dtw = []

for i in range(n_mini_batch_size):
    print(i, end='\r')
    x = data['x'][i].squeeze()
    
    y_duplicate = duplicates['x'][i].squeeze()
    duplicates_dtw.append(dtwalign.dtw(x, y_duplicate, dist=DISTANCE, open_end=False, dist_only=True).normalized_distance)

    y_non_duplicate = non_duplicates['x'][i].squeeze()
    non_duplicates_dtw.append(dtwalign.dtw(x, y_non_duplicate, dist=DISTANCE, open_end=False, dist_only=True).normalized_distance)    

In [ ]:
bins = np.linspace(0, 100., 100)
fig, axes = plt.subplots(2, 1, figsize=(15,10))

duplicates_dtw_filtered = np.array(duplicates_dtw)[np.where(np.array(duplicates_dtw) < 100.)]
non_duplicates_dtw_filtered = np.array(non_duplicates_dtw)[np.where(np.array(non_duplicates_dtw) < 100.)]


sns.distplot(duplicates_dtw,  bins=bins, ax=axes[0], kde=False)
axes[0].title.set_text('duplicates DTW')

sns.distplot(non_duplicates_dtw, bins=bins,  color='y', ax=axes[1], kde=False)
axes[1].title.set_text('non-duplicates DTW')
plt.show()

### KL Divergence between two DTW distributions:

In [ ]:
hist_dup, _ = np.histogram(duplicates_dtw)
hist_dup = hist_dup / np.sum(hist_dup)

hist_non_dup, _ = np.histogram(non_duplicates_dtw)
hist_non_dup = hist_non_dup / np.sum(hist_non_dup)

dkl = scipy.special.kl_div(hist_dup+1e-5, hist_non_dup+1e-5).sum()
print(f"DKL is: {dkl:.4f}")


### As it can be seen, DTW calculated on MFCCs signals failed to capture different and similar samples showing no significant difference between duplicates and non-duplicates pairs. Kullback-Leibler divergence just proves this fact that P (distribution of distances between duplicate samples) and Q (non-duplicate ones) have no significant difference.

# Improving DTW using latent variables from LSTM classifier as a signal

In [ ]:
# load nn

nn_fname = r'C:/Study/SpeechAcquisitionModel/reports/seamise_net_09_25_2019_03_11_PM/net_0.5.net'

nn = torch.load(nn_fname)

In [ ]:
PREFIX_LENS = [50, 40, 30, 20]

for PREFIX_LEN in PREFIX_LENS:
    nn_input = torch.from_numpy(data['x'][:, :PREFIX_LEN, :]).cuda().float()


    z, *_ = nn.single_forward(nn_input)
    z = z.detach().cpu().numpy()

    duplicates_z, *_ = nn.single_forward(torch.from_numpy(duplicates['x']).cuda().float())
    duplicates_z = duplicates_z.detach().cpu().numpy()

    non_duplicates_z, *_ = nn.single_forward(torch.from_numpy(non_duplicates['x']).cuda().float())
    non_duplicates_z = non_duplicates_z.detach().cpu().numpy()

    #########################################################################
    # Show example
    #########################################################################

    i = np.random.randint(data['y'].shape[0])
    sample_idx = i

    fig, axes = plt.subplots(2, 3, figsize=(15,10))

    # plt.figure()
    im = axes[0][0].imshow(z[i].T, vmin=-1, vmax=1)
    # fig.colorbar(im)
    label = index_to_word[data['y'][i]]
    axes[0][0].title.set_text(f'{label}')

    axes[0][1].imshow(duplicates_z[i].T, vmin=-1, vmax=1)
    # plt.colorbar()
    label = index_to_word[data['y'][i]]
    axes[0][1].title.set_text(f'{label}')


    axes[0][2].imshow(non_duplicates_z[i].T, vmin=-1, vmax=1)
    # plt.colorbar()
    label = index_to_word[non_duplicates['y'][i]]
    axes[0][2].title.set_text(f'{label}')

    # DTW

    x = z[sample_idx]

    # self dtw (extpected to be 0)

    y = z[sample_idx]
    res = dtwalign.dtw(x, y, dist=DISTANCE, open_end=True)
    res.plot_path(ax=axes[1][0])
    axes[1][0].set_title(f"anchor DTW = {res.normalized_distance:.4f}")


    # positive dtw

    y = duplicates_z[sample_idx]
    res = dtwalign.dtw(x, y, dist=DISTANCE, open_end=True)
    res.plot_path(ax=axes[1][1])
    axes[1][1].set_title(f"positive DTW = {res.normalized_distance:.4f}")


    # negative dtw

    y = non_duplicates_z[sample_idx]
    res = dtwalign.dtw(x, y, dist=DISTANCE, open_end=True)
    res.plot_path(ax=axes[1][2])
    axes[1][2].set_title(f"negative DTW = {res.normalized_distance:.4f}")

    plt.show()

    ###########################################################################
    # DTW
    ###########################################################################

    ##########################################################################
    #OPEN END = TRUE
    ##########################################################################

    # initialize dist_lists

    duplicates_dtw_z = []
    non_duplicates_dtw_z = []

    duplicates_bmis = []
    non_duplicates_bmis = []

    for i in range(n_mini_batch_size):
        print(i, end='\r')
        x = z[i].squeeze()

        y_duplicate = duplicates_z[i].squeeze()
        d = dtwalign.dtw(x, y_duplicate, dist=DISTANCE, open_end=True)
        duplicates_dtw_z.append(d.normalized_distance)
        duplicates_bmis.append(d.path[-1, -1])

        y_non_duplicate = non_duplicates_z[i].squeeze()
        d = dtwalign.dtw(x, y_non_duplicate, dist=DISTANCE, open_end=True)
        non_duplicates_dtw_z.append(d.normalized_distance)
        non_duplicates_bmis.append(d.path[-1, -1])

    # Show DTW

    bins = np.linspace(min(np.min(duplicates_dtw_z), np.min(non_duplicates_dtw_z)),
                       max(np.max(duplicates_dtw_z), np.max(non_duplicates_dtw_z)),
                       100)
    fig, axes = plt.subplots(1, 1, figsize=(15,8))

    duplicates_bmis = np.array(duplicates_bmis)
    non_duplicates_bmis = np.array(non_duplicates_bmis)


    sns.distplot(duplicates_dtw_z, bins=bins, ax=axes, label="Positives")
    axes.title.set_text(f'DTW (open_end = True) | prefix_len={PREFIX_LEN}')

    sns.distplot(non_duplicates_dtw_z, bins=bins, color='y', ax=axes, label='Negatives')
    axes.legend()
    # axes[1].title.set_text('non-duplicates DTW')
    plt.show()

    hist_dup, _ = np.histogram(duplicates_dtw_z, bins=bins)
    hist_dup = hist_dup / np.sum(hist_dup)

    hist_non_dup, _ = np.histogram(non_duplicates_dtw_z, bins=bins)
    hist_non_dup = hist_non_dup / np.sum(hist_non_dup)

    dkl = scipy.special.kl_div(hist_dup+1e-5, hist_non_dup+1e-5).sum()
    print(f"DKL is: {dkl:.4f}")

    # plot matching segment length

    bins = np.linspace(0, 50., 50)
    fig, axes = plt.subplots(1, 1, figsize=(15,8))

    sns.distplot(duplicates_bmis, ax=axes,  label="Positives", kde=False, bins=bins)
    axes.title.set_text(f'Matching segment length | prefix_len={PREFIX_LEN}')

    sns.distplot(non_duplicates_bmis,   color='y', ax=axes, label='Negatives', kde=False, bins=bins)
    axes.legend()
    plt.show()

    ##########################################################################
    # OPEN END = False
    ##########################################################################

    # initialize dist_lists

    duplicates_dtw_z = []
    non_duplicates_dtw_z = []

    duplicates_bmis = []
    non_duplicates_bmis = []

    for i in range(n_mini_batch_size):
        print(i, end='\r')
        x = z[i].squeeze()

        y_duplicate = duplicates_z[i].squeeze()
        d = dtwalign.dtw(x, y_duplicate, dist=DISTANCE, open_end=False)
        duplicates_dtw_z.append(d.normalized_distance)
        duplicates_bmis.append(d.path[-1, -1])

        y_non_duplicate = non_duplicates_z[i].squeeze()
        d = dtwalign.dtw(x, y_non_duplicate, dist=DISTANCE, open_end=False)
        non_duplicates_dtw_z.append(d.normalized_distance)
        non_duplicates_bmis.append(d.path[-1, -1])


    bins = np.linspace(min(np.min(duplicates_dtw_z), np.min(non_duplicates_dtw_z)),
                       max(np.max(duplicates_dtw_z), np.max(non_duplicates_dtw_z)),
                       100)
    fig, axes = plt.subplots(1, 1, figsize=(15,8))

    sns.distplot(duplicates_dtw_z, bins=bins, ax=axes, label="Positives")
    axes.title.set_text(f'DTW (open_end = False) | prefix_len={PREFIX_LEN}')
 
    sns.distplot(non_duplicates_dtw_z, bins=bins, color='y', ax=axes, label='Negatives')
    axes.legend()
    # axes[1].title.set_text('non-duplicates DTW')
    plt.show()

    hist_dup, _ = np.histogram(duplicates_dtw_z, bins=bins)
    hist_dup = hist_dup / np.sum(hist_dup)

    hist_non_dup, _ = np.histogram(non_duplicates_dtw_z, bins=bins)
    hist_non_dup = hist_non_dup / np.sum(hist_non_dup)

    dkl = scipy.special.kl_div(hist_dup+1e-5, hist_non_dup+1e-5).sum()
    print(f"DKL is: {dkl:.4f}")


